
## Introduction to Verython
Complex structures such as neural networks comprise high-dimensional arrays and a large number of operations. Manually writing our model in Verilog would have been impossible under the three week time constraints--doing so would require tens of thousands of lines of handtyped Verilog. Instead, we wrote a Python library to write the Verilog for us. 

***[Verython](https://github.com/jfw225/verython)*** (portmanteau of Verilog and Python pronounced "verithon") is a Python to Verilog transpiler which generates Verilog modules from objects created in Python. Additionally, it interfaces directly with the free version of Intel's ModelSim to compile the Verilog, simulate the module's waveforms, and export that data back to Python where it compares the simulated data with the expected data computed in Python.


<!-- container: dark -->
#### Syntax 
We start with the notion that lines of code in Verilog (or most languages for that matter) are simply sentences, and we can represent a sentence in Python as a `string`. Thus, if we can generate strings, we can generate Verilog. As we get into the structures in the library, you may notice the use of some types. These types are all defined in **Appendix A: Verython**, and we encourage the reader to reference the appendix in the case of confusion.


##### V_Block
The most fundamental object in ***Verython*** is the `V_Block` object:

In [ ]:
# python
class V_Block(List[str]):
    """
    The type representing a block of verilog code tabbed to the relative level.
    """

    def __init__(self, *lines: List[str]):
        super().__init__([V_Line(line) for line in lines])

A `V_Block` object is essentially a list of `string` objects where each `string` is some line of Verilog code. For instance, the following `V_Block` instance


In [ ]:
# python
width = 4
V_Block(
    f"wire [{width - 1}:0] w;",
    # `*` unpacks each of the strings in the list
    *[f"assign w[{i}] = 1'b1;" for i in range(width)]
)

generates the following verilog code

In [ ]:
# verilog
wire [3:0] w;
assign w[0] = 1'b1;
assign w[1] = 1'b1;
assign w[2] = 1'b1;
assign w[3] = 1'b1;

Obviously, this alone is nothing special, but it provides an easy, readible way to group lines of code and rapidly generate a large amount of Verilog. 


##### Core Blocked Syntax
Since Verilog can be represented by a group of `string` objects, we can represent each of Verilog's core blocked syntax as `V_Block` objects. Here we will show the ***Verython*** version of Verilog Always and If/Else statements, but the rest of the core blocked syntax can be seen in **Appendix A: Verython**.

In [ ]:
# python
def V_Always(
    edge: V_PosEdge or V_NegEdge,
    signal: V_Expression or V_Port or V_Variable
):

    def build(*lines: V_Block) -> V_Block:
        return V_Block(
            f"always @ ({edge()} {signal.name}) begin",
            *[f"\t{line}" for line in lines],
            "end"
        )

    return build
    
def V_If(
    predicate: V_Expression or V_ObjectBase
):

    assert isinstance(predicate, (V_Expression, V_ObjectBase)
                      ), f'"{predicate}" is not a valid predicate.'

    def build(*lines: Iterable[V_Line]) -> V_Block:
        return V_Block(
            f"if ({predicate}) begin",
            *[f"\t{line}" for line in lines],
            "end"
        )

    return build


def V_Else(
    *lines: Iterable[V_Line]
) -> V_Block:

    return V_Block(
        f"else begin",
        *[f"\t{line}" for line in lines],
        "end"
    )

All of the blocked syntax overloads return `V_Block` object, and since a `V_Block` object inherits all of the properties of a Python list, we can use the `*` operator to unpack a `V_Block` into another `V_Block`. For instance, the following two examples are equivalent.

In [ ]:
# python
V_Block(
    "wire clk;",
    "reg [3:0] w;",
    *V_Always(V_Posedge, "clk")(
        *V_If("w == 4'd0")(
            "w <= 4'd1;"
        )
    )
    
) # is equivalent to

V_Block(
    "wire clk;",
    "reg [3:0] w;",
    "".join(*V_Always(V_Posedge, "clk")(
        "".join(V_If("w == 4'd0")(
            "w <= 4'd1;"
        ))
    ))
)

Generated Verilog:

In [ ]:
# verilog
wire clk;
reg [3:0] w;
always @(posedge clk) begin
    if (w == 4'd0) begin
        w <= 4'd1;
    end
end

All of ***Verython*** syntax can be combined in `V_Block` objects to create complex, powerful structures that would be very tedious and cumbersome to manually write in Verilog. 


<!-- container: default -->
#### Objects 
Up until now, you may have noticed that number, wire, and register definitions usage were manually typed in Python `string` objects. However, this disappears with the introduction of ***Verython*** objects.

##### V_Int and V_FixedPoint

In [ ]:
# python
class V_Int(V_Expression):
    """ full implementation not shown """

class V_FixedPoint(V_Expression):
    """ full implementation not shown """

Instances of `V_Int` and `V_FixedPoint` objects keep track of the required bit widths. This is used by ***Verython*** to convert these instances to the proper Verilog format and add an additional layer of error checking before compilation. In addition, Python allows the developer to overload each of the standard library operators for an object. We leveraged this to make the following equivalence possible.

In [ ]:
# python
a = V_Int(1, width=4)
b = V_FixedPoint(1, int_width=1, dec_width=3)

V_Block(
    "wire [3:0] a, b;",
    f"assign a = {a};", 
    f"assign b = {b};"
) # is equivalent to 

V_Block(
    "wire [3:0] a, b;",
    "assign a = 4'b0001;",
    "assign b = 4'b1_000;"
)

Generated Verilog:

In [ ]:
# verilog
wire [3:0] a, b;
assign a = 4'd1;
assign b = 4'b1_000;


##### V_Port, V_Variable, and V_Array

In [ ]:
# python
class V_ObjectBase:
    """ full implementation not shown """

class V_Port(V_ObjectBase):
    """ full implementation not shown """

class V_Variable(V_ObjectBase):
    """ full implementation not shown """

class V_Array(V_ObjectBase, metaclass=V_ArrayMeta):
    """ full implementation not shown """

***Verython*** also has an object representation of Verilog ports, wires, registers, and arrays. Similar to `V_Int` and `V_FixedPoint`, each of the ***Verython*** objects defined above keep track of bit widths and have their default Python operators overloaded. This enables the  ***Verython*** to make the following transcompilation:


In [ ]:
# python
a = V_Int(1, width=4)
b = V_FixedPoint(1, int_width=1, dec_width=3)

pclk = V_Port(module=None,
              port_type=V_Input,
              name="clk")
preset = V_Port(module=None,
                port_type=V_Input,
                name="reset")

pcounter = V_Port(module=None,
              port_type=V_Output,
              dtype=V_Reg,
              width=2,
              name="counter")

wa = V_Variable(module=None,
                dtype=V_Wire,
                width=4,
                name="example_wire")

rb = V_Variable(module=None,
                dtype=V_Reg,
                width=4
                signed=True,
                name="example_reg")

arc = V_Array(module=None,
              dtype=V_RegArray,
              width=4,
              size=2,
              name="example_array")

V_Block(
    "module count_to_four(",
    pclk, preset,
    pcounter,
    ");",
    wa, rb, arc,

    *V_Always(V_PosEdge, pclk)(
        *V_If(preset)(
            pcounter.set(0)

            rb.set(0),

            arc[0].set(a),
            arc[1].set(b),

        ), *V_Else(
            pcounter.set(pcounter + 1),

            rb.set(rb + 1),

            arc[rb].set(wa)
        ),

        *V_If(rb + 1 == 2)(
            rb.set(0)
        )
    ),

    wa.set(rb),
    "endmodule"
) 

Generated Verilog:

In [ ]:
# verilog
module count_to_four(
    input clk, reset,
    output reg [1:0] counter
);
    wire [3:0] example_wire;
    reg signed [3:0] example_reg;
    reg [3:0] example_array [1:0];

    always @(posedge clk) begin
        if (reset) begin
            counter <= 2'd0;
            
            example_reg <= 4'd0;

            example_array[0] <= 4'd1;
            example_array[1] <= 4'b1_000;
        end else begin
            counter <= counter + 2'd1;

            example_reg <= example_reg + 4'd1;

            example_array[example_reg] <= example_wire;
        end

        if (example_reg + 4'd1 == 4'd2) begin
            example_reg <= 4'd0;
        end
    end

    assign example_wire = example_reg;

endmodule


<!-- container: dark -->
#### Modules

If you were at all concerned that you were going to have to manually write out the module header and footer, let me put your mind at ease. ***Verython*** also enables you to write Verilog modules entirely in Python. For the full implementation, look at **Appendix A: Verython**.

[V_Module](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vmodule.py)

In [ ]:
# python
class V_Module:
    """
    The base class for a verilog module.
    """

class V_Iterable(V_Module):
    """
    An object that implements an iterable verilog data structure. 

    Attributes:
        `clk` -- 1-bit input signal
            The port connecting this module to the clock line.

        `reset` -- 1-bit input signal
            The port that enables the caller to reset this module. 

        `write_en` -- 1-bit input signal
            When this signal is HIGH, the data loaded onto `write_data` is 
            stored in memory at address `write_addr`.

        `read_addr` -- (ceil(log2(size)))-bit input signal
            The address from which data is read.

        `write_addr` -- (ceil(log2(size)))-bit input signal
            The address to which data is written.

        `read_data` -- (width)-bit output signal
            The data line into which data is read from memory at address 
            `read_addr`.

        `write_data` -- (width)-bit input signal
            The data line which is written to memory at address `write_addr`.
    """

class V_Target(V_Module):
    """
    The implementation of a module that can be used as the target 
    function in any module. 

    Attributes:
        `clk` -- 1-bit input signal
            The port connecting this module to the clock line.

        `reset` -- 1-bit input signal
            The port that enables the caller to reset this module. 

        `valid` -- 1-bit input signal
            The port that indicates whether or not the input data is valid. The
            caller can manipulate this signal to indicate whether or not the 
            input data is valid.

        `done` -- 1-bit output signal
            The port indicating whether or not the target function has finished 
            computation for a given input. When this flag is `HIGH`, the output 
            data is valid. Thus, the target module should raise this signal 
            when it finishes its task.

        `ready` -- 1-bit output signal
            The port indicating whether or not the target module is ready 
            to begin a task. 


As an example, we will show a ***Verython*** implementation of an M10K block.

In [ ]:
# python
class M10K(V_Iterable):

    def __init__(
        self,
        width: BitWidth,
        size: ArraySize,
        **kwargs,
    ):
        super().__init__(width, size, **kwargs)

        self.memory = V_Array(self, V_RegArray, self.width,
                              self.size, name="memory")

        self.syn_style = '/* synthesis ramstyle = "no_rw_check, M10K" */'

    def generate(self):

        mem_fmt_base, *_ = self.memory.define().split(";")

        return V_Block(
            "// force M10K ram style",
            f'{mem_fmt_base} {self.syn_style} ;'
            "\n",
            *V_Always(V_PosEdge, self.clk)(
                *V_If(self.write_en)(
                    self.memory.set(self.write_addr, self.write_data)
                ),
                self.read_data.set(self.memory.get(self.read_addr))
            )
        )

M10K(width=16, size=784).generate()

Generated Verilog from an instance of `M10K` with `width=16` and `size=784`:

In [ ]:
# verilog
module M10K(
	input    clk,
	input    write_enable,
	input   [9:0] read_addr,
	input   [9:0] write_addr,
	output reg  [15:0] read_data,
	input   [15:0] write_data
);
	// force M10K ram style
	reg  [15:0] memory [783:0]  /* synthesis ramstyle = "no_rw_check, M10K" */;

	always @ (posedge clk) begin
		if (write_enable) begin
			memory[write_addr] <= write_data;
		end
		read_data <= memory[read_addr];
	end
endmodule


<!-- container: default -->
#### States and State Machines
The final piece of the architecture that we'll discuss in this elaborate introduction is ***Verython*** state machines. 


##### V_State

In [ ]:
# python
class V_State(metaclass=_V_State_Meta):
    """
    The object representing a state in a verilog state machine.
    """

    def __init__(self, state_id: V_StateID, width: BitWidth) -> None:
        self.state_id = state_id
        self.width = width

    def generate(self, m: V_Module) -> V_Block:
        """
        This function comprises the state logic: both what is done in the state
        and how the state transitions. The return value should be some verilog
        code and at least one state. The state machine will iterate through
        the returned list of code and replace each `Type[V_State]` with an
        assignment to change the state value. This should be overloaded.

        If no state is found, the next state will be `V_StDone` and the
        module's `done` flag will be raised.

        The parameter `module` is the `V_Module` object in which this state
        will be used.
        """

        return V_Block(
            *V_If(V_Expression(format_int(V_High, 1)))(
                V_State
            )
        )


class V_StateMachine:
    """ 
    Verilog implementation of a finite state machine.
    """

    def __init__(
        self,
        reset_state: Type[V_State],
        *states: Iterable[Type[V_State]]
    ) -> None:

        # determine bit width needed for a state variable
        # (+ 1 is for `V_StDone`)
        self.width = BitWidth(ceil(log2(len(states) + 1)))

        # set the reset state
        self._reset_state = reset_state(
            state_id=V_StateID(0), width=self.width)

        # maps class types to initialized objects
        self._state_map: Dict[NetName, V_State] = {
            str(state): state(state_id=V_StateID(i), width=self.width)
            for i, state in enumerate(states)
        }

    def generate(
        self,
        module: V_Module,
        clk: V_Clock,
        reset: V_Reset,
        done: V_Done,
        edge: Optional[V_PosEdge or V_NegEdge] = V_PosEdge
    ) -> V_Block:
        """
        This function generates a verilog state machine from `self._state_map` 
        that operates on the edge `edge of clock `clk`.

        Each state has a `generate` function that should return an iterable 
        containing lines of verilog and at least `V_State` object. If no 
        `V_State` object is found, an error will be thrown.  

        This function will create a state variable `state` in the verilog 
        module `module`, and will assign each of the states a value. We will 
        initialize `state` to `self._start_state` when `reset` is HIGH. 

        Let `V_i` denote the value assigned to the `i`-th state `S_i`. Then this 
        function will search through the iterables returned from each of the 
        `V_State.generate` calls and replace each `S_i` with `state <= V_i`.

        The state machine will continue to run until `V_StDone` is 
        reached--at which point, the state machine will raise `done` and idle 
        until `reset` is set.
        """


States in any sort of finite state machine usually have two roles: do something and transition to a state. 

The "do something" component is accomplished by providing each state with the module `m` that implements the state machine. Doing so gives the state access to all of the modules ports, variables, and instances. 

The transition part is a little tricky. Suppose when you make your state machine, you want some state $a$ to transition to some other state $b$, and once you get to $b$, you want to go back to $a$. This is implemented in the following pseudocode:

In [ ]:
# python
class StateA:
    def __init__(self, state_b):
        self.state_b = state_b

    def transition(self):
        do_something()

        go_to_state(self.state_b)

class StateB:
    def __init__(self, state_a):
        self.state_a = state_a

    def transition(self):
        do_something()

        go_to_state(self.state_a)

state_a = StateA(state_b=None)
state_b = StateB(state_a=state_a)

Well $a$ and $b$ are really referring to instances of some state object. Suppose you create $a$ first. Well $a$ needs to have a reference to $b$, but $b$ hasn't been instansiated yet. Same logic applies if you create $b$ first. We solve this issue by instead referencing states by their static type rather than their instance. And then at transpile-time, the `V_StateMachine` object creates some `state` register, assigns each referenced state type some number `n`, and replaces the static reference with `state <= n` inside of an always block. Our example above becomes:

In [ ]:
# python
class OnReset(V_State):
    def generate(self, m: V_Module) -> V_Block(

        return V_Block(
            do_something(),

            StateA
        )
    )

class StateA(V_State):
    def generate(self, m: V_Module) -> V_Block(

        return V_Block(
            do_something(),

            StateB
        )
    )

class StateB(V_State):
    def generate(self, m: V_Module) -> V_Block(

        return V_Block(
            do_something(),

            StateA
        )
    )

# notice the state parameters are not instances, they are static types
V_StateMachine(OnReset, StateA, StateB).generate()

Generated Verilog:

In [ ]:
# verilog
reg [1:0] state;

always @(posedge clk) begin
    if (reset) begin
        // do something

        // go to StateA
        state <= 2'd0;
    end
    else begin 
        case (state)
            // StateA
            2'd0: begin
                    // do something

                    // go to StateB
                    state <= 2'd1;
                end
            
            // StateB
            2'd1: begin
                    // do something 

                    // go to StateA
                    state <= 2'd0;
                end
        endcase
    end
end

Combining the tools provided in the ***Verython*** library enables the rapid creation of robust and expansive Verilog code. In fact, let's talk about how we used it to do just that.